In [1]:
import numpy as np
from pyspark.sql import SparkSession
from pyspark.sql import functions as f
from pyspark.sql.types import *
# import os.path
# from os import path

import tensorflow as tf
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
# from tensorflow.keras.models import Sequential
# from tensorflow.keras.layers import LSTM, Dropout, Dense
# from tensorflowonspark import TFCluster
# from sklearn.preprocessing import MinMaxScaler

from pyspark.ml.linalg import Vectors, VectorUDT, DenseVector
from pyspark.sql.functions import udf 

from pyspark.ml.feature import VectorAssembler, StringIndexer, MinMaxScaler


2024-06-27 22:44:07.940283: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 AVX_VNNI FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2024-06-27 22:44:08.262478: I tensorflow/core/util/port.cc:104] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2024-06-27 22:44:09.663878: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer.so.7'; dlerror: libnvinfer.so.7: cannot open shared object file: No such file or directory
2024-06-27 22:44:09.664022: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not l

In [2]:
# Khởi tạo Spark Session
spark = SparkSession.builder.appName("kafka").getOrCreate()
sc = spark.sparkContext

your 131072x1 screen size is bogus. expect trouble
24/06/27 22:44:14 WARN Utils: Your hostname, LAPTOP-VD1TNFDJ resolves to a loopback address: 127.0.1.1; using 10.255.255.254 instead (on interface lo)
24/06/27 22:44:14 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
24/06/27 22:44:15 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


# 1.Bus

## 1.1 Load data

In [156]:
bus = spark.read.csv("../data/clean_data/bus_clean.csv", inferSchema=True, header=True)
bus.toPandas()

,date,route,time,day,location,incident,min_delay,min_gap,direction,vehicle,day_month,month,hour,min
0,2023-01-01,69,2024-06-10 02:34:00,Sunday,warden station,Security,22,44,S,8407,1,1,2,34
1,2023-01-01,35,2024-06-10 03:06:00,Sunday,jane station,Cleaning - Unsanitary,30,60,N,1051,1,1,3,6
2,2023-01-01,900,2024-06-10 03:14:00,Sunday,kipling station,Security,17,17,unknown,3334,1,1,3,14
3,2023-01-01,52,2024-06-10 04:25:00,Sunday,lawrence and yonge,Emergency Services,30,60,E,3520,1,1,4,25
4,2023-01-01,24,2024-06-10 04:35:00,Sunday,danforth and main,Cleaning - Unsanitary,20,40,W,8404,1,1,4,35
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
49046,2023-12-31,960,2024-06-10 00:24:00,Sunday,steeles and village ga,Mechanical,10,20,E,7118,31,12,0,24
49047,2023-12-31,41,2024-06-10 01:26:00,Sunday,keele and eglinton,Operations - Operator,28,53,N,1133,31,12,1,26
49048,2023-12-31,94,2024-06-10 01:31:00,Sunday,castle frank station,Emergency Services,10,20,unknown,8415,31,12,1,31
49049,2023-12-31,63,2024-06-10 01:40:00,Sunday,oakwood and vaughan,Diversion,33,43,N,7010,31,12,1,40


In [157]:
bus_drop = bus.drop("date", "time")

In [158]:
bus_drop.toPandas().info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 49051 entries, 0 to 49050
Data columns (total 12 columns):
 #   Column     Non-Null Count  Dtype 
---  ------     --------------  ----- 
 0   route      49051 non-null  int32 
 1   day        49051 non-null  object
 2   location   49051 non-null  object
 3   incident   49051 non-null  object
 4   min_delay  49051 non-null  int32 
 5   min_gap    49051 non-null  int32 
 6   direction  49051 non-null  object
 7   vehicle    49051 non-null  int32 
 8   day_month  49051 non-null  int32 
 9   month      49051 non-null  int32 
 10  hour       49051 non-null  int32 
 11  min        49051 non-null  int32 
dtypes: int32(8), object(4)
memory usage: 3.0+ MB


## 1.2 StringIndexer col categorical

In [159]:
categorical_cols = ["day", "incident", "direction", "location"]

In [160]:
indexed_data = bus_drop

In [161]:
for col in categorical_cols:
    string_indexer = StringIndexer(inputCol=col, outputCol=col + "_indexed")
    indexed_data = string_indexer.fit(indexed_data).transform(indexed_data)

In [162]:
indexed_data.toPandas().info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 49051 entries, 0 to 49050
Data columns (total 16 columns):
 #   Column             Non-Null Count  Dtype  
---  ------             --------------  -----  
 0   route              49051 non-null  int32  
 1   day                49051 non-null  object 
 2   location           49051 non-null  object 
 3   incident           49051 non-null  object 
 4   min_delay          49051 non-null  int32  
 5   min_gap            49051 non-null  int32  
 6   direction          49051 non-null  object 
 7   vehicle            49051 non-null  int32  
 8   day_month          49051 non-null  int32  
 9   month              49051 non-null  int32  
 10  hour               49051 non-null  int32  
 11  min                49051 non-null  int32  
 12  day_indexed        49051 non-null  float64
 13  incident_indexed   49051 non-null  float64
 14  direction_indexed  49051 non-null  float64
 15  location_indexed   49051 non-null  float64
dtypes: float64(4), int32(8

## 1.3 W2v col direction

In [163]:
maxlen = max(len(s.split()) for s in indexed_data.toPandas()['location'])
print(maxlen)

7


In [164]:
tokenizer = Tokenizer(num_words=10000)
tokenizer.fit_on_texts(indexed_data.toPandas()['location']) 
sequences = tokenizer.texts_to_sequences(indexed_data.toPandas()['location'])
station_w2v = pad_sequences(sequences, maxlen=7) 
station_w2v_list = [x.tolist() for x in station_w2v]

# Create the DataFrame
station_w2v_df = spark.createDataFrame(station_w2v_list, ArrayType(IntegerType())).toDF("location_w2v")

indexed_data = indexed_data.withColumn("id", f.monotonically_increasing_id())
station_w2v_df = station_w2v_df.withColumn("id", f.monotonically_increasing_id())
indexed_data = indexed_data.join(station_w2v_df, "id").drop("id")

# indexed_data = indexed_data.withColumn('station_w2v', f.lit(station_w2v_list))

In [165]:
indexed_data.toPandas().info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 6144 entries, 0 to 6143
Data columns (total 17 columns):
 #   Column             Non-Null Count  Dtype  
---  ------             --------------  -----  
 0   route              6144 non-null   int32  
 1   day                6144 non-null   object 
 2   location           6144 non-null   object 
 3   incident           6144 non-null   object 
 4   min_delay          6144 non-null   int32  
 5   min_gap            6144 non-null   int32  
 6   direction          6144 non-null   object 
 7   vehicle            6144 non-null   int32  
 8   day_month          6144 non-null   int32  
 9   month              6144 non-null   int32  
 10  hour               6144 non-null   int32  
 11  min                6144 non-null   int32  
 12  day_indexed        6144 non-null   float64
 13  incident_indexed   6144 non-null   float64
 14  direction_indexed  6144 non-null   float64
 15  location_indexed   6144 non-null   float64
 16  location_w2v       6144 

## 1.4 Vector and scala feature col

In [175]:
feature_cols = [col for col, dtype in indexed_data.dtypes if dtype in ["int", "bigint", "float", "double"]]
feature_cols.remove("min_delay")
feature_cols

['route',
 'min_gap',
 'vehicle',
 'day_month',
 'month',
 'hour',
 'min',
 'day_indexed',
 'incident_indexed',
 'direction_indexed',
 'location_indexed']

In [176]:
# with location encoding
vector_assembler = VectorAssembler(inputCols=feature_cols, outputCol="features")
assembled_data = vector_assembler.transform(indexed_data)

scaler = MinMaxScaler(inputCol="features", outputCol="scaled_features")
scaler_model = scaler.fit(assembled_data)
scaled_data = scaler_model.transform(assembled_data)

In [168]:
feature_cols.remove('location_indexed')

In [177]:
# with location w2v
list_to_vector_udf = udf(lambda l: Vectors.dense(l), VectorUDT())

scaled_data = scaled_data.withColumn("location_w2v_vector", list_to_vector_udf("location_w2v"))

feature_cols = ["location_w2v_vector"] + feature_cols
vector_assembler = VectorAssembler(inputCols=feature_cols, outputCol="features_w2v")
assembled_data_w2v = vector_assembler.transform(scaled_data)

In [178]:
assembled_data_w2v.toPandas()['features_w2v'][0]

DenseVector([0.0, 0.0, 0.0, 0.0, 0.0, 19.0, 2.0, 69.0, 44.0, 8407.0, 1.0, 1.0, 2.0, 34.0, 6.0, 2.0, 1.0, 8.0])

In [181]:
scaler = MinMaxScaler(inputCol="features_w2v", outputCol="scaled_features_w2v")
scaler_model = scaler.fit(assembled_data_w2v)
scaled_data_w2v = scaler_model.transform(assembled_data_w2v)

to_dense = udf(lambda v: DenseVector(v.toArray()), VectorUDT())

scaled_data_w2v = scaled_data_w2v.withColumn("scaled_features_w2v", to_dense(scaled_data_w2v["scaled_features_w2v"]))

In [152]:
scaled_data_w2v

DataFrame[route: int, day: string, location: string, incident: string, min_delay: int, min_gap: int, direction: string, vehicle: int, day_month: int, month: int, hour: int, min: int, day_indexed: double, incident_indexed: double, direction_indexed: double, location_indexed: double, location_w2v: array<int>, features: vector, scaled_features: vector, location_w2v_vector: vector, features_w2v: vector, scaled_features_w2v: vector]

## 1.6 Save data featured

In [183]:
scaled_data_w2v.toPandas().info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 6144 entries, 0 to 6143
Data columns (total 22 columns):
 #   Column               Non-Null Count  Dtype  
---  ------               --------------  -----  
 0   route                6144 non-null   int32  
 1   day                  6144 non-null   object 
 2   location             6144 non-null   object 
 3   incident             6144 non-null   object 
 4   min_delay            6144 non-null   int32  
 5   min_gap              6144 non-null   int32  
 6   direction            6144 non-null   object 
 7   vehicle              6144 non-null   int32  
 8   day_month            6144 non-null   int32  
 9   month                6144 non-null   int32  
 10  hour                 6144 non-null   int32  
 11  min                  6144 non-null   int32  
 12  day_indexed          6144 non-null   float64
 13  incident_indexed     6144 non-null   float64
 14  direction_indexed    6144 non-null   float64
 15  location_indexed     6144 non-null   f

In [184]:
scaled_data_w2v.toPandas().to_csv('../data/features_data/bus_feature.csv', index=False)

# 2.Subway

## 2.1 Load data

In [19]:
subway = spark.read.csv("../data/clean_data/subway_cleaned.csv", inferSchema=True, header=True)


In [20]:
subway_drop = subway.drop("date", "time")
subway_drop.toPandas()

,day,station,code,min_delay,min_gap,bound,line,vehicle,day_month,month,hour,min,at_station
0,Sunday,museum station,MUPAA,3,9,S,YU,5931,1,1,2,22,1
1,Sunday,vaughan mc station,TUNOA,6,12,S,YU,5451,1,1,8,2,1
2,Sunday,greenwood station,MUNOA,5,10,E,BD,0,1,1,8,11,1
3,Sunday,keele station,TUNOA,5,10,W,BD,0,1,1,8,15,1
4,Sunday,greenwood station,TUNOA,5,10,E,BD,0,1,1,8,16,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...
7377,Thursday,glencairn station,SUUT,14,19,S,YU,5761,30,11,22,37,1
7378,Thursday,bloor yonge station,MUSAN,5,10,S,YU,5666,30,11,23,9,1
7379,Thursday,sheppard yonge station,MUATC,5,10,N,YU,6066,30,11,1,57,1
7380,Thursday,bessarion station,TUML,7,0,E,SHP,6186,30,11,6,29,1


In [21]:
subway_drop.toPandas().info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 7382 entries, 0 to 7381
Data columns (total 13 columns):
 #   Column      Non-Null Count  Dtype 
---  ------      --------------  ----- 
 0   day         7382 non-null   object
 1   station     7382 non-null   object
 2   code        7382 non-null   object
 3   min_delay   7382 non-null   int32 
 4   min_gap     7382 non-null   int32 
 5   bound       7382 non-null   object
 6   line        7382 non-null   object
 7   vehicle     7382 non-null   int32 
 8   day_month   7382 non-null   int32 
 9   month       7382 non-null   int32 
 10  hour        7382 non-null   int32 
 11  min         7382 non-null   int32 
 12  at_station  7382 non-null   int32 
dtypes: int32(8), object(5)
memory usage: 519.2+ KB


## 2.2 StringIndexer col categorical

In [22]:
categorical_cols = ["day", "code", "bound", "line", "station"]

In [23]:
indexed_data = subway_drop

In [24]:
for col in categorical_cols:
    string_indexer = StringIndexer(inputCol=col, outputCol=col + "_indexed")
    indexed_data = string_indexer.fit(indexed_data).transform(indexed_data)

## 2.3 W2v cal station

In [26]:
maxlen = max(len(s.split()) for s in indexed_data.toPandas()['station'])
print(maxlen)

4


In [27]:
tokenizer = Tokenizer(num_words=10000)
tokenizer.fit_on_texts(indexed_data.toPandas()['station']) 
sequences = tokenizer.texts_to_sequences(indexed_data.toPandas()['station'])
station_w2v = pad_sequences(sequences, maxlen=4) 
station_w2v_list = [x.tolist() for x in station_w2v]

# Create the DataFrame
station_w2v_df = spark.createDataFrame(station_w2v_list, ArrayType(IntegerType())).toDF("station_w2v")

indexed_data = indexed_data.withColumn("id", f.monotonically_increasing_id())
station_w2v_df = station_w2v_df.withColumn("id", f.monotonically_increasing_id())
indexed_data = indexed_data.join(station_w2v_df, "id").drop("id")

# indexed_data = indexed_data.withColumn('station_w2v', f.lit(station_w2v_list))

In [28]:
indexed_data.toPandas().info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 461 entries, 0 to 460
Data columns (total 19 columns):
 #   Column           Non-Null Count  Dtype  
---  ------           --------------  -----  
 0   day              461 non-null    object 
 1   station          461 non-null    object 
 2   code             461 non-null    object 
 3   min_delay        461 non-null    int32  
 4   min_gap          461 non-null    int32  
 5   bound            461 non-null    object 
 6   line             461 non-null    object 
 7   vehicle          461 non-null    int32  
 8   day_month        461 non-null    int32  
 9   month            461 non-null    int32  
 10  hour             461 non-null    int32  
 11  min              461 non-null    int32  
 12  at_station       461 non-null    int32  
 13  day_indexed      461 non-null    float64
 14  code_indexed     461 non-null    float64
 15  bound_indexed    461 non-null    float64
 16  line_indexed     461 non-null    float64
 17  station_indexed 

## 2.4 Vector and scala feature col

In [29]:
feature_cols = [col for col, dtype in indexed_data.dtypes if dtype in ["int", "bigint", "float", "double"]]
feature_cols.remove("min_delay")
# feature_cols.append("station_w2v")



In [30]:
# with location encoding
vector_assembler = VectorAssembler(inputCols=feature_cols, outputCol="features")
assembled_data = vector_assembler.transform(indexed_data)

scaler = MinMaxScaler(inputCol="features", outputCol="scaled_features")
scaler_model = scaler.fit(assembled_data)
scaled_data = scaler_model.transform(assembled_data)

In [31]:
feature_cols.remove('station_indexed')

In [32]:
# with location w2v
list_to_vector_udf = udf(lambda l: Vectors.dense(l), VectorUDT())

scaled_data = scaled_data.withColumn("station_w2v_vector", list_to_vector_udf("station_w2v"))

feature_cols = ["station_w2v_vector"] + feature_cols
vector_assembler = VectorAssembler(inputCols=feature_cols, outputCol="features_w2v")
assembled_data_w2v = vector_assembler.transform(scaled_data)

In [33]:
scaler = MinMaxScaler(inputCol="features_w2v", outputCol="scaled_features_w2v")
scaler_model = scaler.fit(assembled_data_w2v)
scaled_data_w2v = scaler_model.transform(assembled_data_w2v)

## 2.6 Save data featured

In [34]:
scaled_data_w2v.toPandas().info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 461 entries, 0 to 460
Data columns (total 24 columns):
 #   Column               Non-Null Count  Dtype  
---  ------               --------------  -----  
 0   day                  461 non-null    object 
 1   station              461 non-null    object 
 2   code                 461 non-null    object 
 3   min_delay            461 non-null    int32  
 4   min_gap              461 non-null    int32  
 5   bound                461 non-null    object 
 6   line                 461 non-null    object 
 7   vehicle              461 non-null    int32  
 8   day_month            461 non-null    int32  
 9   month                461 non-null    int32  
 10  hour                 461 non-null    int32  
 11  min                  461 non-null    int32  
 12  at_station           461 non-null    int32  
 13  day_indexed          461 non-null    float64
 14  code_indexed         461 non-null    float64
 15  bound_indexed        461 non-null    flo

In [35]:
scaled_data_w2v.toPandas().to_csv('../data/features_data/subway_feature.csv', index=False)